In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import zscore



In [ ]:
import pyarrow.parquet as pq

In [ ]:
mda_project1=pd.read_parquet("aed_locations.parquet.gzip")
mda_project2=pd.read_parquet("interventions2.parquet.gzip")
mda_project3=pd.read_parquet("interventions3.parquet.gzip")
mda_project4=pd.read_parquet("interventions_bxl.parquet.gzip")
mda_project5=pd.read_parquet("interventions_bxl2.parquet.gzip")
mda_project7=pd.read_parquet("ambulance_locations.parquet.gzip")
mda_project6=pd.read_parquet("cad9.parquet.gzip")
mda_project8=pd.read_parquet("mug_locations.parquet.gzip")
mda_project9=pd.read_parquet("pit_locations.parquet.gzip")
mda_project10=pd.read_parquet("interventions1.parquet")

In [ ]:

# Print the data types

print(mda_project8.head())

In [ ]:
mda_project6.head(5)

In [ ]:
mda_project1.columns

In [ ]:
mda_project2.columns

In [ ]:
mda_project3.columns

In [ ]:
mda_project4.columns

In [ ]:
mda_project6.columns

In [ ]:
mda_project7.columns

In [ ]:
mda_project8.columns

In [ ]:
mda_project9.columns

In [ ]:
print(mda_project6['EventType Trip'].unique().tolist())

In [ ]:
print(mda_project10['EventType Firstcall'].unique().tolist())

In [ ]:
print(mda_project2['EventType Firstcall'].unique().tolist())

In [ ]:
print(mda_project3['EventType Firstcall'].unique().tolist())


In [ ]:
# Join interventions1_cardiac, interventions2_cardiac, and interventions3_cardiac into a single dataframe
interventions_cardiac = pd.concat([mda_project10, mda_project2, mda_project3])

In [ ]:
# Rename columns for consistency
mda_project6.rename(columns={'Mission ID':'Mission ID', 'Service Name':'Service Name', 'Latitude permanence': 'Latitude Permanence', 'Longitude permanence': 'Longitude Permanence', 'Latitude intervention': 'Latitude Intervention', 'Longitude intervention': 'Longitude Intervention', 'Vector Type':'Vector Type'}, inplace=True)
interventions_cardiac.rename(columns={'Mission ID':'Mission ID', 'Service Name':'Service Name', 'Latitude permanence': 'Latitude Permanence', 'Longitude permanence': 'Longitude Permanence', 'Latitude intervention': 'Latitude Intervention', 'Longitude intervention': 'Longitude Intervention', 'Vector type':'Vector Type', 'Abandon reason':'Abandon Reason'}, inplace=True)
mda_project5.rename(columns={'Mission ID':'Mission ID', 'Service Name NL':'Service Name', 'Latitude permanence': 'Latitude Permanence', 'Longitude permanence': 'Longitude Permanence', 'Latitude intervention': 'Latitude Intervention', 'Longitude intervention': 'Longitude Intervention', 'Vector type NL':'Vector Type', 'Abandon reason NL':'Abandon Reason'}, inplace=True)
mda_project4.rename(columns={'mission_id':'Mission ID', 'service_name':'Service Name', 'latitude_permanence': 'Latitude Permanence', 'longitude_permanence':'Longitude Permanence', 'latitude_intervention':'Latitude Intervention', 'longitude_intervention':'Longitude Intervention', 'vector_type':'Vector Type', 't0':'T0', 't1':'T1', 't2':'T2', 't3':'T3', 't4':'T4', 't5':'T5', 't6':'T6', 'abandon_reason':'Abandon Reason'}, inplace=True)
#print(interventions_cardiac)

In [ ]:
interventions_cardiac.head()

In [ ]:
# Convert date/time strings to pandas datetime objects (this operation is realized in several step, dependent on the time stamp, due to the non-uniformity of the data even within the different time stamps)
mda_project6['T0'] = pd.to_datetime(mda_project6['T0'], format='%Y-%m-%d %H:%M:%S.%f')
interventions_cardiac['T0'] = pd.to_datetime(interventions_cardiac['T0'], format='%d%b%y:%H:%M:%S')
mda_project5['T0'] = pd.to_datetime(mda_project5['T0'], format='%d%b%y:%H:%M:%S')
mda_project4['T0'] = mda_project4['T0'].astype(str).str[:19]
mda_project4['T0'] = pd.to_datetime(mda_project4['T0'], format='%Y-%m-%d %H:%M:%S')

mda_project6['T1'] = pd.to_datetime(mda_project6['T1'], format='%d%b%y:%H:%M:%S')
interventions_cardiac['T1'] = pd.to_datetime(interventions_cardiac['T1'], format='%d%b%y:%H:%M:%S')
mda_project5['T1'] = pd.to_datetime(mda_project5['T1'], format='%d%b%y:%H:%M:%S')
mda_project4['T1'] = mda_project4['T1'].astype(str).str[:19]
mda_project4['T1'] = pd.to_datetime(mda_project4['T1'], format='%Y-%m-%d %H:%M:%S', errors = 'coerce')

times_steps = ['T2', 'T3', 'T4', 'T5', 'T6']
for time_step in times_steps:
    mda_project6[time_step] = mda_project6[time_step].astype(str).str[:19]
    mda_project6[time_step] = pd.to_datetime(mda_project6[time_step], format='%Y-%m-%d %H:%M:%S', errors= 'coerce')
    interventions_cardiac[time_step] = interventions_cardiac[time_step].astype(str).str[:19]
    interventions_cardiac[time_step] = pd.to_datetime(interventions_cardiac[time_step], format='%Y-%m-%d %H:%M:%S', errors ='coerce')
    mda_project5[time_step] = pd.to_datetime(mda_project5[time_step], format='%d%b%y:%H:%M:%S')
    mda_project4[time_step] = mda_project4[time_step].astype(str).str[:19]
    mda_project4[time_step] = pd.to_datetime(mda_project4[time_step], format='%Y-%m-%d %H:%M:%S', errors = 'coerce')

# Add a new column 'Abandon Reason' to cad9_cardiac with value 'Overleden', and also overwrite the interventions_bxl2_cardiac with the same (maybe some issue with the filtering earlier??)
mda_project6['Abandon Reason'] = 'Overleden'
mda_project5['Abandon Reason'] = 'Overleden'

# Add a new column to track the original file
mda_project6['Original File'] = 'cad9_cardiac'
interventions_cardiac['Original File'] = 'interventions_cardiac'
mda_project4['Original File'] = 'mda_project4'
mda_project5['Original File'] = 'mda_project5'

# Concatenate all dataframes
frames = [
    mda_project6[['Mission ID', 'Service Name', 'Latitude Permanence', 'Longitude Permanence', 'Latitude Intervention', 'Longitude Intervention', 'Vector Type', 'T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'Abandon Reason', 'Original File']], 
    interventions_cardiac[['Mission ID', 'Service Name', 'Latitude Permanence', 'Longitude Permanence', 'Latitude Intervention', 'Longitude Intervention', 'Vector Type', 'T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'Abandon Reason', 'Original File']], 
    mda_project4[['Mission ID', 'Service Name', 'Latitude Permanence', 'Longitude Permanence', 'Latitude Intervention', 'Longitude Intervention', 'Vector Type', 'T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'Abandon Reason', 'Original File']], 
   mda_project5[['Mission ID', 'Service Name', 'Latitude Permanence', 'Longitude Permanence', 'Latitude Intervention', 'Longitude Intervention', 'Vector Type', 'T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'Abandon Reason', 'Original File']]
]
cardiac_interventions = pd.concat(frames)

# Replace values in the 'Vector Type' column
cardiac_interventions['Vector Type'] = cardiac_interventions['Vector Type'].replace({'Ambulance': 'AMB', 'Ambulance Event': 'AMB', 'MUG Event': 'MUG'})

## Correction of the latitude and longitude values
# Define the desired ranges
latitude_range = (48, 53)
longitude_range = (1, 8)

# Define a function to adjust latitude and longitude values
def adjust_coordinate(value, coordinate_range):
    if value < coordinate_range[0] or value > coordinate_range[1]:
        return value / 10
    return value

# Apply the adjustment function to Latitude Permanence and Latitude Intervention
while any((cardiac_interventions['Latitude Permanence'] < latitude_range[0]) | (cardiac_interventions['Latitude Permanence'] > latitude_range[1])):
    cardiac_interventions['Latitude Permanence'] = cardiac_interventions['Latitude Permanence'].apply(lambda x: adjust_coordinate(x, latitude_range))
    cardiac_interventions['Latitude Intervention'] = cardiac_interventions['Latitude Intervention'].apply(lambda x: adjust_coordinate(x, latitude_range))

# Apply the adjustment function to Longitude Permanence and Longitude Intervention
while any((cardiac_interventions['Longitude Permanence'] < longitude_range[0]) | (cardiac_interventions['Longitude Permanence'] > longitude_range[1])):
    cardiac_interventions['Longitude Permanence'] = cardiac_interventions['Longitude Permanence'].apply(lambda x: adjust_coordinate(x, longitude_range))
    cardiac_interventions['Longitude Intervention'] = cardiac_interventions['Longitude Intervention'].apply(lambda x: adjust_coordinate(x, longitude_range))

In [ ]:
cardiac_interventions['T3_T0_difference_minutes'] = (cardiac_interventions['T3'] - cardiac_interventions['T0']).dt.total_seconds() / 60

In [ ]:
cardiac_interventions.head()

In [ ]:
#Analyzing response times by service type????
response_times_by_service = cardiac_interventions.groupby('Service Name')['T3_T0_difference_minutes'].mean()

# Plotting
sns.barplot(x=response_times_by_service.index, y=response_times_by_service.values)
plt.ylabel('Average Response Time (minutes)')
plt.title('Average Response Time by Service Type')
plt.xticks(rotation=30)
plt.show()

In [ ]:
# Convert latitude and longitude to floats
cardiac_interventions['Latitude Intervention'] = pd.to_numeric(cardiac_interventions['Latitude Intervention'], errors='coerce')
cardiac_interventions['Longitude Intervention'] = pd.to_numeric(cardiac_interventions['Longitude Intervention'], errors='coerce')

# Drop rows where coordinates are NaN
cardiac_interventions = cardiac_interventions.dropna(subset=['Latitude Intervention', 'Longitude Intervention'])

In [ ]:
# Group by location and count the occurrences
location_counts = cardiac_interventions.groupby(['Latitude Intervention', 'Longitude Intervention']).size().reset_index(name='Counts')

# Sort to find top locations with most events
top_locations = location_counts.sort_values(by='Counts', ascending=False).head(10)
print(top_locations)

In [ ]:
pip install folium

In [ ]:
import folium

In [ ]:
# Create a base map
map = folium.Map(location=[cardiac_interventions['Latitude Intervention'].mean(), cardiac_interventions['Longitude Intervention'].mean()], zoom_start=12)

# Add markers for top hotspot locations
for idx, row in top_locations.iterrows():
    folium.CircleMarker([row['Latitude Intervention'], row['Longitude Intervention']],
                        radius=row['Counts']/10, # this scales the circle sizes by count
                        popup=f"Count: {row['Counts']}",
                        color='red',
                        fill=True,
                        fill_color='red').add_to(map)

# Display the map
map

In [ ]:
# hypothesis: longer response time is associated to high likelhood of death
# Convert abandonment to binary 
cardiac_interventions['abandoned'] = cardiac_interventions['Abandon Reason'].notna()  # Assumes NaN where no abandonment

# Calculate correlation
correlation = cardiac_interventions['T3_T0_difference_minutes'].corr(cardiac_interventions['abandoned'])
print("Correlation between response time and abandonment:", correlation)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Convert abandonment reason to a binary variable
cardiac_interventions['Abandon Binary'] = cardiac_interventions['Abandon Reason'].apply(lambda x: 1 if x == 'Overleden' else 0)

# Features and target
X = cardiac_interventions[['T3_T0_difference_minutes']].notna()  # or more features can be included
y = cardiac_interventions['Abandon Binary']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# First, split into training + validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Then split the training + validation set into separate training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

In [ ]:

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

In [ ]:
# Create a logistic regression classifier
classifier = LogisticRegression(random_state = 42)
# Fit the classifier to the training data
classifier.fit(X_train, y_train)

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
# making confusion matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')